In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from pathlib import Path
import os
from tensorflow.keras.applications import ResNet50V2
tf.__version__

'2.4.1'

In [2]:
PATH = '/kaggle/input/'

os.listdir(PATH)

['dataset-melanoma224', 'melanoma224']

In [3]:
train = pd.read_csv(f'{PATH}/dataset-melanoma224/subset.csv')
val = pd.read_csv(f'{PATH}/dataset-melanoma224/val_split.csv')
train.shape, val.shape


((2220, 8), (10932, 8))

In [4]:
BATCH_SIZE = 64
AUTO = tf.data.experimental.AUTOTUNE # Permite obtener el numero de cpu 

def decode(name, label):
    img = tf.io.read_file(name)
    img = tf.image.decode_jpeg(img,channels=3)
    img = tf.cast(img, tf.float32)
    return img,label


def load_ds(df):
    # Le indicamos a tensorflow que da igual el orden de lectura de las imagenes, lo que aumentará la velocidad
    options = tf.data.Options()
    options.experimental_deterministic = False
    #---------------
    imgs , labels = df["image_name"].values, df["target"].values
    imgs = [f'{PATH}/melanoma224/jpeg224/train/{name}.jpg' for name in imgs]
    ds = tf.data.Dataset.from_tensor_slices((imgs,labels))
    ds = ds.with_options(options) # Aplicamos las opciones que hemos puesto
    ds = ds.map(decode, num_parallel_calls=AUTO) # num_parallel_calls permitira a la funcion map procesar en paralelo tantas imagenes como le pongamos
                                                 # maximo el numero de cpu que tenga el ordenador por eso usamo la variable AUTO
    df = ds.cache() # Cada epochs se leen las imagenes y es preferible cargar nuestro datasets en cache
    ds = ds.shuffle(2048)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size = AUTO) # Mientras la gpu esta calculando la cpu vaya leyendo imagenes
    return ds


In [5]:
train_ds = load_ds(train)
val_ds = load_ds(train)


In [6]:
IMAGE_SIZE = (224,224,3)

encoder = ResNet50V2(
    include_top=False,
    input_shape=IMAGE_SIZE,
    weights='imagenet'
)
encoder.trainable=False

inputs = keras.Input(shape=IMAGE_SIZE)
x = keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = encoder(x, training = False)
x = keras.layers.GlobalAveragePooling2D()(x)
ouputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, ouputs)
model.summary()


94674944/94668760 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
rescaling (Rescaling)        (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


In [7]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.AUC(name="auc")]
)

In [11]:
filepath = f"kaggle/working/checkpoints/checkpoint"
cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    monitor="val_auc",
    verbose=1,
    save_best_only=True,
    save_weights_only=True, # Guardar todo el modelo o solo los pesos
    mode="max", # Guardara el valor maximo del parametro que monitorizamos, en caso de loss poner "min"
)

In [12]:
model.fit(train_ds,
          epochs=10,
          validation_data=val_ds,
          validation_steps=10,
          callbacks=[cb])
# Hemos pasado de 20 segundos a 8 segundos por epochs por los cambios introducidos en la funcion load_ds

Epoch 1/10
35/35 [==============================] - 8s 185ms/step - loss: 0.0499 - auc: 0.9490 - val_loss: 0.0462 - val_auc: 0.9732

Epoch 00001: val_auc improved from -inf to 0.97319, saving model to kaggle/working/checkpoints/checkpoint
Epoch 2/10
35/35 [==============================] - 8s 178ms/step - loss: 0.0478 - auc: 0.9547 - val_loss: 0.0395 - val_auc: 0.9918

Epoch 00002: val_auc improved from 0.97319 to 0.99177, saving model to kaggle/working/checkpoints/checkpoint
Epoch 3/10
35/35 [==============================] - 8s 187ms/step - loss: 0.0468 - auc: 0.9587 - val_loss: 0.0484 - val_auc: 0.9843

Epoch 00003: val_auc did not improve from 0.99177
Epoch 4/10
35/35 [==============================] - 8s 176ms/step - loss: 0.0443 - auc: 0.9659 - val_loss: 0.0273 - val_auc: 0.9913

Epoch 00004: val_auc did not improve from 0.99177
Epoch 5/10
35/35 [==============================] - 8s 181ms/step - loss: 0.0448 - auc: 0.9621 - val_loss: 0.0507 - val_auc: 0.9501

Epoch 00005: val_auc

In [ ]:
model.load_weights(filepath)# Cargamos los pesos de la mejor epochs val_auc 0.99177
model.save("model.h5")#Guardamos todo el modelo pero con los mejores pesos


In [ ]:

test = pd.read_csv(f'{PATH}/melanoma224/test.csv')
BATCH_SIZE = 64

def decode_test(name):
    img = tf.io.read_file(name)
    img = tf.image.decode_jpeg(img,channels=3)
    img = tf.cast(img, tf.float32)
    return img


def load_test_ds(df):
    imgs  = df["image_name"].values
    imgs = [f'{PATH}/melanoma224/jpeg224/test/{name}.jpg' for name in imgs]
    ds = tf.data.Dataset.from_tensor_slices(imgs)
    ds = ds.map(decode_test)
    ds = ds.batch(BATCH_SIZE)
    return ds

In [ ]:
test_ds = load_test_ds(test)


In [ ]:
preds = model.predict(test_ds)


In [ ]:
submission = pd.DataFrame({
    'image_name':test['image_name'].values,
    'target':preds.ravel()
})

In [ ]:
submission.to_csv("/kaggle/working/submission.csv", index=False)
